In [2]:
import warnings
warnings.filterwarnings('ignore')
import pyterrier as pt
if not pt.java.started():
    pt.java.init()

import os

# sys.path.append('/nfs/primary/exposure-fairness')
os.chdir('/nfs/primary/exposure-fairness')
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 100)
import matplotlib.pyplot as plt

Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


In [3]:
os.getcwd()

'/nfs/primary/exposure-fairness'

In [4]:
# data_dir = '/nfs/datasets/cxj/retrievability-bias/data/bm25'
# df = pd.read_csv(f'{data_dir}/df_bm25_dev_0.csv', index_col=0).reset_index()

# data_dir = '/nfs/datasets/cxj/retrievability-bias/data/bm25_monot5'
# df = pd.read_csv(f'{data_dir}/df_bm25_monot5_dev_0.csv', index_col=0).reset_index()

# data_dir = '/nfs/datasets/cxj/retrievability-bias/data/rtr_splade'
# df = pd.read_csv(f'{data_dir}/df_rtr_splade_dev_topics_0.csv', index_col=0).reset_index()

# data_dir = '/nfs/datasets/cxj/retrievability-bias/data/tctcolbert'
# df = pd.read_csv(f'{data_dir}/df_tctcolbert_dev_0.csv', index_col=0).reset_index()

# data_dir = '/nfs/datasets/cxj/retrievability-bias/data/bm25_tctcolbert'
# df = pd.read_csv(f'{data_dir}/df_bm25_tctcolbert_dev_0.csv', index_col=0).reset_index()

# csv = '/mnt/datasets/cxj/exposure-fairness/bm25/results.pkl'
# df = pd.read_pickle(csv).reset_index()

# csv = '/mnt/datasets/cxj/exposure-fairness/bm25/df_bm25_dev_0.csv'
# df = pd.read_csv(csv, index_col=0).reset_index()

# csv = '/mnt/datasets/cxj/exposure-fairness/bm25_monot5/df_bm25_monot5_dev_0.csv'
# df = pd.read_csv(csv, index_col=0).reset_index()

# csv = '/mnt/datasets/cxj/exposure-fairness/splade/df_splade_dev_0.csv'
# df = pd.read_csv(csv, index_col=0).reset_index()

# csv = '/mnt/datasets/cxj/exposure-fairness/tctcolbert/df_tctcolbert_dev_0.csv'
# df = pd.read_csv(csv, index_col=0).reset_index()

# csv = '/mnt/datasets/cxj/exposure-fairness/bm25_tctcolbert/df_bm25_tctcolbert_dev_0.csv'
# df = pd.read_csv(csv, index_col=0).reset_index()

pkl = '/nfs/datasets/cxj/exposure-fairness/v1/results.pkl'
df = pd.read_pickle(pkl).reset_index(drop=True)

In [ ]:
df[(df.modelname == 'bm25') & (df.num_groups == 2000)]

In [16]:
result = '/nfs/datasets/cxj/exposure-fairness/v1/df_bm25_dev_0.csv'
df = pd.read_csv(result,index_col=0).reset_index(drop=True)
df.head()

,docid,docno,rank,score,query,text
0,7187236,7187236,0,67.528593,cost of endless pools swim spa,The Endless Pool 15' endless pools swim spa Endless Pool Nightmare Buy A Big Spa Instead Great t...
1,7471198,7471198,1,62.121928,cost of endless pools swim spa,Cal Spas is the leading maker of home resort products. Cal Spas makes a great jetted swim spa wi...
2,5365326,5365326,2,51.958492,cost of endless pools swim spa,"How much does a swim spa cost. 1 A top of the line, branded swim spa 9 feet by 16 feet with the..."
3,7187241,7187241,3,51.947947,cost of endless pools swim spa,Disclaimer: Endless Pools (and all other swim spa manufacturers) are intentionally secretive abo...
4,6802210,6802210,4,47.925985,cost of endless pools swim spa,"Endless Pools is one of the first companies to develop this Swim Spa, or “endless” swimming pool..."


In [ ]:
# df2 = pd.DataFrame()
# df2['query_id'] = df['qid'].astype(str)
# df2['doc_id'] = df['docno'].astype(str)
# df2['score'] = df['score']
# ir_measures.calc_aggregate([RR, nDCG@10], qrels, df2)

In [20]:
df = pd.read_csv('./results/result_all_models_all_groups.csv')
df = df.round(2)

In [21]:
df.head()

,modelname,threshold,grps,min_gini,mean_gini,max_gini,nDCG@10,RR
0,bm25,0,500,0.19,0.29,0.56,NaN,NaN
1,bm25,30,500,0.19,0.30,0.58,NaN,NaN
2,bm25,60,500,0.20,0.33,0.58,NaN,NaN
3,bm25,90,500,0.24,0.40,0.67,NaN,NaN
4,bm25,0,1000,0.18,0.28,0.61,NaN,NaN


In [ ]:
import json

def save(dict, file):
    with open(file,'w') as f:
        json.dump(dict, f)

def load(file):
    with open(file,'r') as f:
        ms_docids = json.load(f)
    return ms_docids

import statistics
def calc_stats(modelname,df,scoredF, topics):
    if not os.path.exists(scoredF):
        init_msmarco_dict = './results/ms_docids.json'
        docids_score = load(init_msmarco_dict)
        for qid in pt.tqdm(topics['qid']):
            D = df[df['qid'] == np.int64(qid)]
            for dno in D['docno']:
                rank = D[D['docno'] == dno]['rank'].values[0]
                score = 100 / np.log(rank + 2)  # plus 2 because the ranks start from zero
                docids_score[str(dno)] += score
        save(docids_score, scoredF)

    docids_score = load(scoredF)
    scores_df = pd.DataFrame.from_dict(docids_score, orient="index", columns=["score"])
    scores_df = scores_df[scores_df["score"] > 0]
    scores = scores_df['score'].to_list()

    mean = statistics.mean(scores)
    std_dev = statistics.stdev(scores)
    gini_value = Gini(scores)
    return mean, std_dev, gini_value